# MLP using k fold and Optuna

## Import Libraries

In [ ]:
# import required libraries
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
import optuna
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier

## Load Dataset

In [ ]:
# read datasets
train = pd.read_csv("../pdata/data/train.csv")
test = pd.read_csv("../pdata/data/test.csv")

In [ ]:
target_feature = train["y"]
train_df = train.drop("y", axis=1)
test_df = test.drop("id", axis=1)

## Balance the train Dataset

In [ ]:
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(train_df,target_feature)

## Scale the features

In [ ]:
# scaling the dataset
scaler = StandardScaler()
scaled_train = scaler.fit_transform(X_train_smote)
scaled_test = scaler.transform(test_df)

# target feature
target = y_train_smote.to_numpy()

## Optuna + k fold

In [ ]:
# optuna
def objective(trial):
    # suggest hyper params
    n_layers = trial.suggest_int('n_layers', 1, 3)
    layers = [trial.suggest_int(f'n_units_l{i}', 16, 128) for i in range(n_layers)]
    activation = trial.suggest_categorical('activation',['relu','tanh'])
    solver = trial.suggest_categorical('solver', ['adam','sgd'])
    alpha = trial.suggest_float('alpha', 1e-6, 1e-2, log=True)
    learning_rate_init = trial.suggest_float('learning_rate_init', 1e-4, 1e-1, log=True)

    # k = 5 fold cross validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    for train_idx, val_idx in kf.split(scaled_train):
        X_tr, X_val = scaled_train[train_idx], scaled_train [val_idx]
        y_tr, y_val = target[train_idx], target[val_idx]

        # MLP model
        clf = MLPClassifier(
            hidden_layer_sizes = tuple(layers),
            activation = activation,
            solver = solver,
            alpha = alpha,
            learning_rate_init = learning_rate_init,
            max_iter = 300,
            random_state = 42
        )
        clf.fit(X_tr, y_tr)
        scores.append(clf.score(X_val, y_val))
    return np.mean(scores)

In [ ]:
# optimize hyper params
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=25)

print("Best params:", study.best_params)
print("Best CV score:", study.best_value)

## Train on Full Data Using Best Params

In [ ]:
# train the final model on full training data with best hyper params
best = study.best_params
layers = [best[f'n_units_l{i}'] for i in range(best['n_layers'])]
final_model = MLPClassifier(
    hidden_layer_size = tuple(layers),
    activation = best['activation'],
    solver = best['solver'],
    alpha = best['alpha'],
    learning_rate_init = best['learning_rate_init'],
    max_iter = 300,
    random_state = 42
)

## Predict on Test Data

In [ ]:
# predict on test set
y_pred = final_model.predict(scaled_test)

In [ ]:
final_y = []
for x in y_pred:
    if x == 1:
        final_y.append(1)
        continue
    final_y.append(0)

In [ ]:
result = pd.DataFrame({
    "id": list(test["id"]),
    "y": final_y
})

In [ ]:
# save the first submission
result.to_csv("../submissions/submission16.csv", index=False)